In [1]:
!conda env list

# conda environments:
#
base                     /home/goswami.p/miniconda3
cs5330_pa5_1             /home/goswami.p/miniconda3/envs/cs5330_pa5_1
dicl                     /home/goswami.p/miniconda3/envs/dicl
ezflow                *  /home/goswami.p/miniconda3/envs/ezflow
flow                     /home/goswami.p/miniconda3/envs/flow
inpaint                  /home/goswami.p/miniconda3/envs/inpaint



In [2]:
!nvidia-smi

Mon Nov 28 08:33:34 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:3B:00.0 Off |                    0 |
| N/A   38C    P0    56W / 300W |      0MiB / 32768MiB |      2%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import math
import time
import torch
import torch.nn as nn
import torch.nn.functional as F

from ezflow.modules import BaseModule
from ezflow.encoder import build_encoder

from torch.nn.functional import pad
from torch.nn.init import trunc_normal_

from timm.models.layers import trunc_normal_, DropPath
from natten.functional import natten2dqkrpb, natten2dav

In [4]:
from nnflow.models.gmflow.transformer import FeatureFlowAttention
from nnflow.models.gmflow.matching import global_correlation_softmax
from nnflow.models.gmflow.position import PositionEmbeddingSine
from nnflow.models.gmflow.nat import ConvTokenizer, ConvDownsampler, Mlp

In [5]:
def count_params(model):
    return str(sum(p.numel() for p in model.parameters() if p.requires_grad) / 1000000) + "M params"

In [6]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

___

## Modified Neighborhood Attention

In [7]:
# Create NATLayer with Cross Attention
# shortcut in NAT == source in GMFlow Feature Transformer

In [7]:
class ModifiedNeighborhoodAttention(nn.Module):
    """
    Neighborhood Attention 2D Module
    """
    def __init__(self, dim, kernel_size, num_heads,
                 qkv_bias=True, qk_scale=None, attn_drop=0., proj_drop=0.,
                 dilation=None):
        super().__init__()
        self.num_heads = num_heads
        self.head_dim = dim // self.num_heads
        self.scale = qk_scale or self.head_dim ** -0.5
        assert kernel_size > 1 and kernel_size % 2 == 1, \
            f"Kernel size must be an odd number greater than 1, got {kernel_size}."
        self.kernel_size = kernel_size
        if type(dilation) is str:
            self.dilation = None
            self.window_size = None
        else:
            assert dilation is None or dilation >= 1, \
                f"Dilation must be greater than or equal to 1, got {dilation}."
            self.dilation = dilation or 1
            self.window_size = self.kernel_size * self.dilation

        # self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.q_proj = nn.Linear(dim, dim, bias=qkv_bias)
        self.k_proj = nn.Linear(dim, dim, bias=qkv_bias)
        self.v_proj = nn.Linear(dim, dim, bias=qkv_bias)
        
        self.rpb = nn.Parameter(torch.zeros(num_heads, (2 * kernel_size - 1), (2 * kernel_size - 1)))
        trunc_normal_(self.rpb, std=.02, mean=0., a=-2., b=2.)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

    def forward(self, q, k, v):
        assert q.shape == k.shape == k.shape
        B, Hp, Wp, C = q.shape
        H, W = int(Hp), int(Wp)
        pad_l = pad_t = pad_r = pad_b = 0
        dilation = self.dilation
        window_size = self.window_size
        if window_size is None:
            dilation = max(min(H, W) // self.kernel_size, 1)
            window_size = dilation * self.kernel_size
        if H < window_size or W < window_size:
            pad_l = pad_t = 0
            pad_r = max(0, window_size - W)
            pad_b = max(0, window_size - H)
            q = pad(q, (0, 0, pad_l, pad_r, pad_t, pad_b))
            k = pad(k, (0, 0, pad_l, pad_r, pad_t, pad_b))
            v = pad(v, (0, 0, pad_l, pad_r, pad_t, pad_b))
            _, H, W, _ = q.shape
        # qkv = self.qkv(x).reshape(B, H, W, 3, self.num_heads, self.head_dim).permute(3, 0, 4, 1, 2, 5)
        # q, k, v = qkv[0], qkv[1], qkv[2]
        q = self.q_proj(q).reshape(B, H, W, self.num_heads, self.head_dim).permute(0, 3, 1, 2, 4)
        k = self.k_proj(k).reshape(B, H, W, self.num_heads, self.head_dim).permute(0, 3, 1, 2, 4)
        v = self.v_proj(v).reshape(B, H, W, self.num_heads, self.head_dim).permute(0, 3, 1, 2, 4)
        
        # print(q.shape, k.shape, v.shape)
        q = q * self.scale
        attn = natten2dqkrpb(q, k, self.rpb, dilation)
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)
        x = natten2dav(attn, v, dilation)
        x = x.permute(0, 2, 3, 1, 4).reshape(B, H, W, C)
        if pad_r or pad_b:
            x = x[:, :Hp, :Wp, :]

        return self.proj_drop(self.proj(x))

    def extra_repr(self) -> str:
        return f'kernel_size={self.kernel_size}, dilation={self.dilation}, head_dim={self.head_dim}, num_heads={self.num_heads}'

In [9]:
patch_embed = ConvTokenizer(in_chans=3, embed_dim=64, norm_layer=nn.LayerNorm)

In [10]:
attn = ModifiedNeighborhoodAttention(
    dim=64,
    kernel_size=7,
    num_heads=2
)

In [11]:
x = torch.randn(1,3,256,256)
out = patch_embed(x)
out.shape

torch.Size([1, 64, 64, 64])

In [12]:
out = attn(out, out, out)
out.shape

torch.Size([1, 64, 64, 64])

In [13]:
attn

ModifiedNeighborhoodAttention(
  kernel_size=7, dilation=1, head_dim=32, num_heads=2
  (q_proj): Linear(in_features=64, out_features=64, bias=True)
  (k_proj): Linear(in_features=64, out_features=64, bias=True)
  (v_proj): Linear(in_features=64, out_features=64, bias=True)
  (attn_drop): Dropout(p=0.0, inplace=False)
  (proj): Linear(in_features=64, out_features=64, bias=True)
  (proj_drop): Dropout(p=0.0, inplace=False)
)

___

In [11]:
# New Attention Layer

In [8]:
class ModifiedNATLayer(nn.Module):
    def __init__(
        self,
        dim,
        num_heads,
        kernel_size=7,
        dilation=None,
        mlp_ratio=4.0,
        qkv_bias=True,
        qk_scale=None,
        drop=0.0,
        attn_drop=0.0,
        drop_path=0.0,
        act_layer=nn.GELU,
        norm_layer=nn.LayerNorm,
        no_ffn=False
    ):
        super().__init__()
        self.dim = dim
        self.num_heads = num_heads
        self.mlp_ratio = mlp_ratio
        self.no_ffn = no_ffn

        self.norm1 = norm_layer(dim)
        self.attn = ModifiedNeighborhoodAttention(
            dim,
            kernel_size=kernel_size,
            dilation=dilation,
            num_heads=num_heads,
            qkv_bias=qkv_bias,
            qk_scale=qk_scale,
            attn_drop=attn_drop,
            proj_drop=drop,
        )

        self.drop_path = DropPath(drop_path) if drop_path > 0.0 else nn.Identity()
        
        # no ffn after self-attn, with ffn after cross-attn
        if not self.no_ffn:
            self.norm2 = norm_layer(dim)
            self.mlp = Mlp(
                in_features=dim,
                hidden_features=int(dim * mlp_ratio),
                act_layer=act_layer,
                drop=drop,
            )


    def forward(self, source, target):
        shortcut = source

        x = torch.cat([source, target], dim=0)
        x = self.norm1(x)

        source, target = x.chunk(chunks=2, dim=0)   
        query, key, value = source, target, target

        x = self.attn(query, key, value)
        x = shortcut + self.drop_path(x)
        
        # no ffn after self-attn, with ffn after cross-attn
        if not self.no_ffn:
            x = x + self.drop_path(self.mlp(self.norm2(x)))
            
        return x
    

In [13]:
attn_layer = ModifiedNATLayer(
    dim=64,
    num_heads=2,
    mlp_ratio=3.0, 
    no_ffn=True
)

In [14]:
attn_layer

ModifiedNATLayer(
  (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
  (attn): ModifiedNeighborhoodAttention(
    kernel_size=7, dilation=1, head_dim=32, num_heads=2
    (q_proj): Linear(in_features=64, out_features=64, bias=True)
    (k_proj): Linear(in_features=64, out_features=64, bias=True)
    (v_proj): Linear(in_features=64, out_features=64, bias=True)
    (attn_drop): Dropout(p=0.0, inplace=False)
    (proj): Linear(in_features=64, out_features=64, bias=True)
    (proj_drop): Dropout(p=0.0, inplace=False)
  )
  (drop_path): Identity()
)

In [15]:
x = torch.randn(1,3,256,256)
out = patch_embed(x)
out.shape

torch.Size([1, 64, 64, 64])

In [16]:
out = attn_layer(source=out, target=out)
out.shape

torch.Size([1, 64, 64, 64])

____

In [17]:
# New Transformer Block with alternating Self-Cross layers

In [9]:
class ModifiedNATBlock(nn.Module):
    def __init__(
        self,
        dim,
        depth,
        num_heads,
        kernel_size,
        dilations=None,
        downsample=True,
        mlp_ratio=4.0,
        qkv_bias=True,
        qk_scale=None,
        drop=0.0,
        attn_drop=0.0,
        drop_path=0.0,
        norm_layer=nn.LayerNorm,
        self_no_ffn=True,
        cross_no_ffn=False
    ):
        super().__init__()
        self.dim = dim
        self.depth = depth

        self.blocks = nn.ModuleList()
        for i in range(depth):
            # Self Attention Block
            self.blocks.append(
                 ModifiedNATLayer(
                    dim=dim,
                    num_heads=num_heads,
                    kernel_size=kernel_size,
                    dilation=None if dilations is None else dilations[i],
                    mlp_ratio=mlp_ratio,
                    qkv_bias=qkv_bias,
                    qk_scale=qk_scale,
                    drop=drop,
                    attn_drop=attn_drop,
                    drop_path=drop_path[i]
                    if isinstance(drop_path, list)
                    else drop_path,
                    norm_layer=norm_layer,
                    no_ffn=self_no_ffn
                )
            )
            
            # Cross Attention Block
            self.blocks.append(
                 ModifiedNATLayer(
                    dim=dim,
                    num_heads=num_heads,
                    kernel_size=kernel_size,
                    dilation=None if dilations is None else dilations[i],
                    mlp_ratio=mlp_ratio,
                    qkv_bias=qkv_bias,
                    qk_scale=qk_scale,
                    drop=drop,
                    attn_drop=attn_drop,
                    drop_path=drop_path[i]
                    if isinstance(drop_path, list)
                    else drop_path,
                    norm_layer=norm_layer,
                    no_ffn=cross_no_ffn
                )
            )
            
        self.downsample = (
            None if not downsample else ConvDownsampler(dim=dim, norm_layer=norm_layer)
        )

    def forward(self, source, target):
        for i in range(len(self.blocks)):
            
            # self attention
            if i % 2 == 0:
                source = self.blocks[i](source, source)
                
            # cross attention and feed forward
            else:
                source = self.blocks[i](source, target)
            
        if self.downsample is None:
            return source
        
        return self.downsample(source)

In [19]:
transformer_blk = ModifiedNATBlock(
    dim=64,
    depth=3,
    num_heads=2,
    kernel_size=7,
    mlp_ratio=3
)

In [22]:
count_params(transformer_blk)

'0.2515M params'

In [23]:
x = torch.randn(1,3,512,512)
out = patch_embed(x)
out.shape

torch.Size([1, 128, 128, 64])

In [24]:
out = transformer_blk(source=out, target=out)
out.shape

torch.Size([1, 64, 64, 128])

___

In [25]:
# New Transformer Encoder

In [10]:
class ModifiedNAT(nn.Module):
    
    def __init__(
        self,
        embed_dim,
        mlp_ratio,
        depths,
        num_heads,
        drop_path_rate=0.2,
        in_chans=3,
        kernel_size=7,
        dilations=None,
        qkv_bias=True,
        qk_scale=None,
        drop_rate=0.0,
        attn_drop_rate=0.0,
        norm_layer=nn.LayerNorm,
        self_no_ffn=True,
        cross_no_ffn=False,
        use_sine_pos_embed=True,
        **kwargs
    ):
        super().__init__()

        
        if isinstance(dilations, str) and dilations == 'None':
            dilations = None 

        self.num_levels = len(depths)
        self.embed_dim = embed_dim
        self.num_features = int(embed_dim * 2 ** (self.num_levels - 1))
        self.mlp_ratio = mlp_ratio
        self.use_sine_pos_embed = use_sine_pos_embed

        self.patch_embed = ConvTokenizer(
            in_chans=in_chans, embed_dim=embed_dim, norm_layer=norm_layer
        )

        self.pos_drop = nn.Dropout(p=drop_rate)

        dpr = [x.item() for x in torch.linspace(0, drop_path_rate, sum(depths))]
        self.levels = nn.ModuleList()
        for i in range(self.num_levels):
            level = ModifiedNATBlock(
                dim=int(embed_dim * 2**i),
                depth=depths[i],
                num_heads=num_heads[i],
                kernel_size=kernel_size,
                dilations=None if dilations is None else dilations[i],
                mlp_ratio=self.mlp_ratio,
                qkv_bias=qkv_bias,
                qk_scale=qk_scale,
                drop=drop_rate,
                attn_drop=attn_drop_rate,
                drop_path=dpr[sum(depths[:i]) : sum(depths[: i + 1])],
                norm_layer=norm_layer,
                downsample=(i < self.num_levels - 1),
                self_no_ffn=self_no_ffn,
                cross_no_ffn=cross_no_ffn
            )
            self.levels.append(level)

        self.norm = norm_layer(self.num_features)
        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=0.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    @torch.jit.ignore
    def no_weight_decay_keywords(self):
        return {"rpb"}
    
    def _add_position_embed(self, img1, img2, feature_channels):
        pos_enc = PositionEmbeddingSine(num_pos_feats=feature_channels // 2)
        position = pos_enc(img1.permute(0,3,1,2))
        
        position = position.permute(0,2,3,1)
        
        img1 = img1 + position
        img2 = img2 + position
        return img1, img2
        
    def forward_features(self, img1, img2):
        b, c, h, w =  img1.shape
        
        x = self.patch_embed(torch.cat([img1,img2], dim=0))  
        x = self.pos_drop(x)
        
        img1, img2 = x.chunk(chunks=2, dim=0)
        
        if self.use_sine_pos_embed:
            img1, img2 = self._add_position_embed(img1, img2, feature_channels=self.embed_dim)
        
        # Concat img1 and img2 in batch dimension to compute in parallel
        concat1 = torch.cat([img1, img2], dim=0) # 2B, H, W, C 
        concat2 = torch.cat([img2, img1], dim=0) # 2B, H, W, C
        
        for level in self.levels:
            concat1 = level(concat1, concat2)
            
            # update feature2
            concat2 = torch.cat(concat1.chunk(chunks=2,dim=0)[::-1], dim=0)
            
        concat1 = self.norm(concat1)
        
        feature1, feature2 = concat1.chunk(chunks=2,dim=0) # B, H, W, C      
        return feature1, feature2

    def forward(self, img1, img2):
        

        feature1, feature2 = self.forward_features(img1, img2)
        
        feature1 = feature1.permute(0,3,1,2)
        feature2 = feature2.permute(0,3,1,2)

        return feature1, feature2

In [29]:
nat = ModifiedNAT(
    embed_dim=64,
    mlp_ratio=3,
    depths=[8,12],
    num_heads=[2,4],
    drop_path_rate=0.2,
    in_chans=3,
    kernel_size=7,
    use_sine_pos_embed=True
)

In [30]:
count_params(nat)

'3.36352M params'

In [32]:
img1 = torch.randn(1,3,368,496)
img2 = torch.randn(1,3,368,496)

feat1, feat2 = nat(img1, img2)

feat1.shape, feat2.shape

(torch.Size([1, 128, 46, 62]), torch.Size([1, 128, 46, 62]))

___

## New End 2 End Transformer

In [11]:
class CrossFlow(BaseModule):
    def __init__(self, cfg):
        
        super(CrossFlow, self).__init__()

        self.cfg = cfg

        self.num_scales = cfg.MODEL.NUM_SCALES
        self.feature_channels = cfg.MODEL.FEATURE_CHANNELS
        self.upsample_factor = cfg.MODEL.UPSAMPLE_FACTOR
        self.num_head = cfg.MODEL.NUM_HEADS
        self.attention_type = cfg.MODEL.ATTENTION_TYPE
        self.ffn_dim_expansion = cfg.MODEL.FFN_DIM_EXPANSION
        self.num_transformer_layers = cfg.MODEL.NUM_TRANSFORMER_LAYERS

        self.attn_splits_list=cfg.MODEL.ATTN_SPLITS_LIST
        self.corr_radius_list=cfg.MODEL.CORR_RADIUS_LIST
        self.prop_radius_list=cfg.MODEL.PROP_RADIUS_LIST
        self.pred_bidir_flow=cfg.MODEL.PRED_BIDIR_FLOW
        

        # Transformer Backbone with alternating self attention and cross attention
        self.backbone = ModifiedNAT(
            embed_dim=cfg.ENCODER.EMBED_DIMS,
            mlp_ratio=cfg.ENCODER.MLP_RATIO,
            depths=cfg.ENCODER.DEPTHS,
            num_heads=cfg.ENCODER.NUM_HEADS,
            drop_path_rate=cfg.ENCODER.DROP_PATH_RATE,
            in_chans=cfg.ENCODER.IN_CHANNELS,
            kernel_size=cfg.ENCODER.KERNEL_SIZE,
            dilations=cfg.ENCODER.DILATIONS,
            self_no_ffn=cfg.ENCODER.SELF_NO_FFN,
            cross_no_ffn=cfg.ENCODER.CROSS_NO_FFN,
            use_sine_pos_embed=cfg.ENCODER.USE_SINE_POS_EMBED
        )

        # flow propagation with self-attn
        self.feature_flow_attn = FeatureFlowAttention(in_channels=self.feature_channels)

        # convex upsampling: concat feature0 and flow as input
        self.upsampler = nn.Sequential(nn.Conv2d(2 + self.feature_channels, 256, 3, 1, 1),
                                       nn.ReLU(inplace=True),
                                       nn.Conv2d(256, self.upsample_factor ** 2 * 9, 1, 1, 0))

    
    def upsample_flow(self, flow, feature, bilinear=False, upsample_factor=8,
                      ):
        if bilinear:
            up_flow = F.interpolate(flow, scale_factor=upsample_factor,
                                    mode='bilinear', align_corners=True) * upsample_factor

        else:
            # convex upsampling
            concat = torch.cat((flow, feature), dim=1)

            mask = self.upsampler(concat)
            b, flow_channel, h, w = flow.shape
            mask = mask.view(b, 1, 9, self.upsample_factor, self.upsample_factor, h, w)  # [B, 1, 9, K, K, H, W]
            mask = torch.softmax(mask, dim=2)

            up_flow = F.unfold(self.upsample_factor * flow, [3, 3], padding=1)
            up_flow = up_flow.view(b, flow_channel, 9, 1, 1, h, w)  # [B, 2, 9, 1, 1, H, W]

            up_flow = torch.sum(mask * up_flow, dim=2)  # [B, 2, K, K, H, W]
            up_flow = up_flow.permute(0, 1, 4, 2, 5, 3)  # [B, 2, K, H, K, W]
            up_flow = up_flow.reshape(b, flow_channel, self.upsample_factor * h,
                                      self.upsample_factor * w)  # [B, 2, K*H, K*W]

        return up_flow

    def forward(self, img0, img1):

        results_dict = {}
        flow_preds = []

        # extract features
        feature0, feature1 = self.backbone(img0, img1)          

        assert len(self.attn_splits_list) == len(self.corr_radius_list) == len(self.prop_radius_list) == self.num_scales


        upsample_factor = self.upsample_factor

        attn_splits = self.attn_splits_list[0]
        corr_radius = self.corr_radius_list[0]
        prop_radius = self.prop_radius_list[0]

            
        # Global matching correlation and softmax
        # when predicting bidirectional flow, flow is the 
        # concatenation of forward flow and backward flow in batch dim [2*B,2,H,W]
        flow = global_correlation_softmax(feature0, feature1, self.pred_bidir_flow)[0]
        

        # upsample to the original resolution for supervison
        if self.training:  # only need to upsample intermediate flow predictions at training time
            flow_bilinear = self.upsample_flow(flow, None, bilinear=True, upsample_factor=upsample_factor)
            flow_preds.append(flow_bilinear)

        # flow propagation with self-attn
        if self.pred_bidir_flow:
            feature0 = torch.cat((feature0, feature1), dim=0)  # [2*B, C, H, W] for propagation
            
        flow = self.feature_flow_attn(feature0, flow.detach(),
                                      local_window_attn=prop_radius > 0,
                                      local_window_radius=prop_radius)


        flow_up = self.upsample_flow(flow, feature0)
        flow_preds.append(flow_up)

        results_dict.update({'flow_preds': flow_preds})

        if not self.training:
            results_dict["flow_upsampled"] = results_dict["flow_preds"][0]

        return results_dict

In [12]:
from ezflow.engine import get_training_cfg as get_cfg

In [35]:
cfg = get_cfg("../configs/gmflow/models/crossflow_v02.yaml")
cfg

CfgNode({'NAME': 'GMFlowV2', 'ENCODER': CfgNode({'NAME': 'NAT', 'IN_CHANNELS': 3, 'DEPTHS': [3, 4], 'NUM_HEADS': [2, 4], 'EMBED_DIMS': 64, 'MLP_RATIO': 3, 'DROP_PATH_RATE': 0.2, 'KERNEL_SIZE': 7, 'DILATIONS': [[1, 8, 1], [1, 4, 1, 4]], 'USE_SINE_POS_EMBED': True, 'SELF_NO_FFN': True, 'CROSS_NO_FFN': False}), 'MODEL': CfgNode({'FEATURE_CHANNELS': 128, 'NUM_SCALES': 1, 'UPSAMPLE_FACTOR': 8, 'NUM_HEADS': 1, 'ATTENTION_TYPE': 'swin', 'FFN_DIM_EXPANSION': 4, 'NUM_TRANSFORMER_LAYERS': 6, 'ATTN_SPLITS_LIST': [2], 'CORR_RADIUS_LIST': [-1], 'PROP_RADIUS_LIST': [-1], 'PRED_BIDIR_FLOW': False})})

In [36]:
model = CrossFlow(cfg)

In [37]:
count_params(model)

'1.684236M params'

In [38]:
img1 = torch.randn(1,3,368,496)
img2 = torch.randn(1,3,368,496)

results = model(img1, img2)
results['flow_preds'][0].shape, results['flow_preds'][1].shape

(torch.Size([1, 2, 368, 496]), torch.Size([1, 2, 368, 496]))

In [79]:
del model

___

## Train the model

In [14]:
import pprint as pp

from ezflow.engine import Trainer
from nnflow import eval_model, CustomDataloaderCreator

In [15]:
model_cfg = get_cfg("../configs/gmflow/models/crossflow_v04.yaml")
training_cfg = get_cfg("../configs/gmflow/trainer/chairs_v2_1.yaml")

training_cfg.DATA.BATCH_SIZE = 10
training_cfg.NUM_STEPS = 50000
training_cfg.SCHEDULER.PARAMS.total_steps = 50100
training_cfg.LOG_DIR = "../../results/crossflow/logs/exp004"
training_cfg.CKPT_DIR = "../../results/crossflow/ckpts/exp004"
training_cfg.FREEZE_BATCH_NORM = False

pp.pprint(training_cfg)

{'CKPT_DIR': '../../results/crossflow/ckpts/exp004',
 'CKPT_INTERVAL': 20000,
 'CRITERION': {'CUSTOM': True,
               'NAME': 'SequenceLoss',
               'PARAMS': CfgNode({'gamma': 0.8, 'max_flow': 400.0})},
 'DATA': {'APPEND_VALID_MASK': True,
          'AUGMENTATION': {'PARAMS': {'TRAINING': {'AUTOFLOW_SPATIAL_PARAMS': {'black': False,
                                                                               'enabled': False,
                                                                               'rot': [0.4,
                                                                                       0.03],
                                                                               'scale': [0.3,
                                                                                         0.03,
                                                                                         0.1],
                                                                               'squ

In [16]:
train_loader_creator = CustomDataloaderCreator(
    batch_size=training_cfg.DATA.BATCH_SIZE,
    num_workers=training_cfg.DATA.NUM_WORKERS,
    pin_memory=training_cfg.DATA.PIN_MEMORY,
    append_valid_mask=training_cfg.DATA.APPEND_VALID_MASK,
    shuffle=training_cfg.DATA.SHUFFLE
)

val_loader_creator = CustomDataloaderCreator(
    batch_size=training_cfg.DATA.BATCH_SIZE,
    num_workers=training_cfg.DATA.NUM_WORKERS,
    pin_memory=training_cfg.DATA.PIN_MEMORY,
    append_valid_mask=training_cfg.DATA.APPEND_VALID_MASK,
    shuffle=training_cfg.DATA.SHUFFLE
)

In [17]:
train_loader_creator.add_FlyingChairs(
    root_dir="../../../../Datasets/FlyingChairs_release/data",
    crop=True,
    crop_type="random",
    crop_size=training_cfg.DATA.TRAIN_CROP_SIZE,
    augment=training_cfg.DATA.AUGMENTATION.USE,
    aug_params={
        "spatial_aug_params": training_cfg.DATA.AUGMENTATION.PARAMS.TRAINING.SPATIAL_AUG_PARAMS,
        "color_aug_params":training_cfg.DATA.AUGMENTATION.PARAMS.TRAINING.COLOR_AUG_PARAMS,
        "eraser_aug_params": training_cfg.DATA.AUGMENTATION.PARAMS.TRAINING.ERASER_AUG_PARAMS,
        "noise_params": training_cfg.DATA.AUGMENTATION.PARAMS.TRAINING.NOISE_PARAMS,
        "spatial_params": training_cfg.DATA.AUGMENTATION.PARAMS.TRAINING.AUTOFLOW_SPATIAL_PARAMS,
        "translate_params": training_cfg.DATA.AUGMENTATION.PARAMS.TRAINING.TRANSLATE_PARAMS,
        "rotate_params": training_cfg.DATA.AUGMENTATION.PARAMS.TRAINING.ROTATE_PARAMS
    },
    norm_params=training_cfg.DATA.NORM_PARAMS
)

In [18]:
val_loader_creator.add_FlyingChairs(
    root_dir="../../../../Datasets/FlyingChairs_release/data",
    split="validation",
    crop=True,
    crop_type="center",
    crop_size=training_cfg.DATA.VAL_CROP_SIZE,
    augment=False,
    norm_params=training_cfg.DATA.NORM_PARAMS
)

In [19]:
model = CrossFlow(model_cfg)

In [20]:
count_params(model)

'3.844352M params'

In [22]:
trainer = Trainer(
    training_cfg, 
    model, 
    train_loader = train_loader_creator.get_dataloader(), 
    val_loader = val_loader_creator.get_dataloader()
)

Total image pairs loaded: 22230/22232

Total image pairs loaded: 640/640



___

## Training output 1

In [ ]:
trainer.train()

Loss function: SequenceLoss is initialized!
Optimizer: AdamW is initialized!
Scheduler: OneCycleLR is initialized!
Training config:

CKPT_DIR: ../../results/crossflow/ckpts/exp001
CKPT_INTERVAL: 20000
CRITERION:
  CUSTOM: True
  NAME: SequenceLoss
  PARAMS:
    gamma: 0.8
    max_flow: 400.0
DATA:
  APPEND_VALID_MASK: True
  AUGMENTATION:
    PARAMS:
      TRAINING:
        AUTOFLOW_SPATIAL_PARAMS:
          black: False
          enabled: False
          rot: [0.4, 0.03]
          scale: [0.3, 0.03, 0.1]
          squeeze: [0.3, 0.0]
          trans: [0.4, 0.03]
        COLOR_AUG_PARAMS:
          aug_prob: 0.2
          brightness: 0.4
          contrast: 0.4
          enabled: True
          hue: 0.15915494309189535
          saturation: 0.4
        ERASER_AUG_PARAMS:
          aug_prob: 0.5
          bounds: [50, 100]
          enabled: True
        NOISE_PARAMS:
          aug_prob: 0.5
          enabled: False
          noise_std_range: 0.06
        ROTATE_PARAMS:
          aug_pr

/home/goswami.p/miniconda3/envs/ezflow/lib/python3.7/site-packages/nnflow-0.0.0-py3.7.egg/nnflow/models/gmflow/position.py:39: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
/home/goswami.p/miniconda3/envs/ezflow/lib/python3.7/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1640811797118/work/aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Iterations: 100, Total iterations: 100, Average batch training loss: 49.07904767990112
Iterations: 200, Total iterations: 200, Average batch training loss: 34.91838252544403
Iterations: 300, Total iterations: 300, Average batch training loss: 30.086434790293374
Iterations: 400, Total iterations: 400, Average batch training loss: 27.542579951286317
Iterations: 500, Total iterations: 500, Average batch training loss: 25.68077686882019
Iterations: 600, Total iterations: 600, Average batch training loss: 24.51905212243398
Iterations: 700, Total iterations: 700, Average batch training loss: 23.689530146462577
Iterations: 800, Total iterations: 800, Average batch training loss: 23.024980469942093
Iterations: 900, Total iterations: 900, Average batch training loss: 22.591071795887416
Iterations: 1000, Total iterations: 1000, Average batch training loss: 22.220978223323822

 --------------------------------------------------------------------------------

Iteration 1000: Average validation los

___

## Training output 2

In [ ]:
trainer.train()

Loss function: SequenceLoss is initialized!
Optimizer: AdamW is initialized!
Scheduler: OneCycleLR is initialized!
Training config:

CKPT_DIR: ../../results/crossflow/ckpts/exp002
CKPT_INTERVAL: 20000
CRITERION:
  CUSTOM: True
  NAME: SequenceLoss
  PARAMS:
    gamma: 0.8
    max_flow: 400.0
DATA:
  APPEND_VALID_MASK: True
  AUGMENTATION:
    PARAMS:
      TRAINING:
        AUTOFLOW_SPATIAL_PARAMS:
          black: False
          enabled: False
          rot: [0.4, 0.03]
          scale: [0.3, 0.03, 0.1]
          squeeze: [0.3, 0.0]
          trans: [0.4, 0.03]
        COLOR_AUG_PARAMS:
          aug_prob: 0.2
          brightness: 0.4
          contrast: 0.4
          enabled: True
          hue: 0.15915494309189535
          saturation: 0.4
        ERASER_AUG_PARAMS:
          aug_prob: 0.5
          bounds: [50, 100]
          enabled: True
        NOISE_PARAMS:
          aug_prob: 0.5
          enabled: False
          noise_std_range: 0.06
        ROTATE_PARAMS:
          aug_pr

/home/goswami.p/miniconda3/envs/ezflow/lib/python3.7/site-packages/nnflow-0.0.0-py3.7.egg/nnflow/models/gmflow/position.py:39: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
/home/goswami.p/miniconda3/envs/ezflow/lib/python3.7/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1640811797118/work/aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Iterations: 100, Total iterations: 100, Average batch training loss: 47.402778425216674
Iterations: 200, Total iterations: 200, Average batch training loss: 34.730524196624756
Iterations: 300, Total iterations: 300, Average batch training loss: 30.00316294034322
Iterations: 400, Total iterations: 400, Average batch training loss: 27.572381217479705
Iterations: 500, Total iterations: 500, Average batch training loss: 25.961023359298707
Iterations: 600, Total iterations: 600, Average batch training loss: 24.760241158803304
Iterations: 700, Total iterations: 700, Average batch training loss: 23.832342450278144
Iterations: 800, Total iterations: 800, Average batch training loss: 23.23447922706604
Iterations: 900, Total iterations: 900, Average batch training loss: 22.79281840854221
Iterations: 1000, Total iterations: 1000, Average batch training loss: 22.250091296195983

 --------------------------------------------------------------------------------

Iteration 1000: Average validation lo

___

## Traning output 3

In [ ]:
trainer.train()

Loss function: SequenceLoss is initialized!
Optimizer: AdamW is initialized!
Scheduler: OneCycleLR is initialized!
Training config:

CKPT_DIR: ../../results/crossflow/ckpts/exp003
CKPT_INTERVAL: 20000
CRITERION:
  CUSTOM: True
  NAME: SequenceLoss
  PARAMS:
    gamma: 0.8
    max_flow: 400.0
DATA:
  APPEND_VALID_MASK: True
  AUGMENTATION:
    PARAMS:
      TRAINING:
        AUTOFLOW_SPATIAL_PARAMS:
          black: False
          enabled: False
          rot: [0.4, 0.03]
          scale: [0.3, 0.03, 0.1]
          squeeze: [0.3, 0.0]
          trans: [0.4, 0.03]
        COLOR_AUG_PARAMS:
          aug_prob: 0.2
          brightness: 0.4
          contrast: 0.4
          enabled: True
          hue: 0.15915494309189535
          saturation: 0.4
        ERASER_AUG_PARAMS:
          aug_prob: 0.5
          bounds: [50, 100]
          enabled: True
        NOISE_PARAMS:
          aug_prob: 0.5
          enabled: False
          noise_std_range: 0.06
        ROTATE_PARAMS:
          aug_pr

/home/goswami.p/miniconda3/envs/ezflow/lib/python3.7/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1640811797118/work/aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Iterations: 100, Total iterations: 100, Average batch training loss: 39.230209560394286
Iterations: 200, Total iterations: 200, Average batch training loss: 30.188817505836486
Iterations: 300, Total iterations: 300, Average batch training loss: 27.124033584594727
Iterations: 400, Total iterations: 400, Average batch training loss: 25.11518214941025
Iterations: 500, Total iterations: 500, Average batch training loss: 23.938732753753662
Iterations: 600, Total iterations: 600, Average batch training loss: 22.988590288162232
Iterations: 700, Total iterations: 700, Average batch training loss: 22.328436083112443
Iterations: 800, Total iterations: 800, Average batch training loss: 21.879796508550644
Iterations: 900, Total iterations: 900, Average batch training loss: 21.528786935806274
Iterations: 1000, Total iterations: 1000, Average batch training loss: 21.203034631729125

 --------------------------------------------------------------------------------

Iteration 1000: Average validation 

___

## Training output 4

In [ ]:
trainer.train()

Loss function: SequenceLoss is initialized!
Optimizer: AdamW is initialized!
Scheduler: OneCycleLR is initialized!
Training config:

CKPT_DIR: ../../results/crossflow/ckpts/exp004
CKPT_INTERVAL: 20000
CRITERION:
  CUSTOM: True
  NAME: SequenceLoss
  PARAMS:
    gamma: 0.8
    max_flow: 400.0
DATA:
  APPEND_VALID_MASK: True
  AUGMENTATION:
    PARAMS:
      TRAINING:
        AUTOFLOW_SPATIAL_PARAMS:
          black: False
          enabled: False
          rot: [0.4, 0.03]
          scale: [0.3, 0.03, 0.1]
          squeeze: [0.3, 0.0]
          trans: [0.4, 0.03]
        COLOR_AUG_PARAMS:
          aug_prob: 0.2
          brightness: 0.4
          contrast: 0.4
          enabled: True
          hue: 0.15915494309189535
          saturation: 0.4
        ERASER_AUG_PARAMS:
          aug_prob: 0.5
          bounds: [50, 100]
          enabled: True
        NOISE_PARAMS:
          aug_prob: 0.5
          enabled: False
          noise_std_range: 0.06
        ROTATE_PARAMS:
          aug_pr

/home/goswami.p/miniconda3/envs/ezflow/lib/python3.7/site-packages/nnflow-0.0.0-py3.7.egg/nnflow/models/gmflow/position.py:39: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
/home/goswami.p/miniconda3/envs/ezflow/lib/python3.7/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1640811797118/work/aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Iterations: 100, Total iterations: 100, Average batch training loss: 38.806310939788816
Iterations: 200, Total iterations: 200, Average batch training loss: 29.86527271270752
Iterations: 300, Total iterations: 300, Average batch training loss: 26.4670383644104
Iterations: 400, Total iterations: 400, Average batch training loss: 24.626682333946228
Iterations: 500, Total iterations: 500, Average batch training loss: 23.34748571586609
Iterations: 600, Total iterations: 600, Average batch training loss: 22.67428199450175
Iterations: 700, Total iterations: 700, Average batch training loss: 22.064218209130424
Iterations: 800, Total iterations: 800, Average batch training loss: 21.582395669817924
Iterations: 900, Total iterations: 900, Average batch training loss: 21.205847839249504
Iterations: 1000, Total iterations: 1000, Average batch training loss: 20.946390319347383

 --------------------------------------------------------------------------------

Iteration 1000: Average validation loss